In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy import linalg
import math
from neicio.readstation import readStation
from neicio.shake import ShakeGrid
from neicio.gmt import GMTGrid
import time
from openquake.hazardlib.imt import from_string
from openquake.hazardlib.site import Site, SiteCollection
from openquake.hazardlib.geo import Point
from openquake.hazardlib.correlation import JB2009CorrelationModel
from openquake.hazardlib.correlation import GA2010CorrelationModel
from openquake.hazardlib.correlation import BaseCorrelationModel
from matplotlib import cm
from Correlation.setup import initialize
from Correlation.loop import main
from Correlation.realizations import realizations

In [18]:
# Variable of interest                                                                                                                                                                                                        
voi = 'PGA'

# Intensity Factor
intensity_factor = 1

# Get shakemap for desired variable, PGA, uncertainty grid and stationdata                                                                                                                                                    
# Selected Stations: Units in pctg                                                                                                                                                                                            
shakemap = ShakeGrid('Inputs/grid.xml', variable = '%s' % voi)

# Uncertainty Data: Units in ln(pctg)                                                                                                                                                                                         
uncertainty = ShakeGrid('Inputs/uncertainty.xml', variable= 'STD%s' % (voi))

# Station Data: Units in pctg                                                                                                                                                                                                 
stationlist = 'Inputs/stationlist.xml'
stationdata = readStation(stationlist)

print np.size(stationdata['lon']), 'stations'


# Sets up grid spacing, site collections, and other data values
# optional parameters: dm, dn, the grid discritization defaulting to 1
variables = initialize(shakemap, uncertainty, stationdata, False)
M = variables['M']
N = variables['N']
K = variables['K']

# Calculate the random array, stored for testing
#rand = np.random.randn(M*N).flatten()
rand = []

with open('rand.txt') as f:
    for line in f:
        rand.append(float(line))
f.close()



0 stations
	Initialization Time: 0.394114017487


In [5]:
R = [15, 25, 35, 30, 32, 35]

f = open('workfileNR26_uncorr.txt', 'w')

f.write('Northridge, %d stations, max_R = %d\n' %(np.size(stationdata['pga']), R[-1]))
s = str(np.size(R))
f.write(s+'\n')
s = str(M)
f.write(s+'\n')
s = str(N)
f.write(s+'\n')

data_NEW = np.zeros([M*N, np.size(R)])

for k in range(0,np.size(R)):
    out = main(variables, R[k], voi, rand, 0.9)
    data_NEW[:,k] = np.reshape(out['cor'], [M*N])
    
    for j in range(0, M*N):
        s = str(data_NEW[j,k])
        f.write(s+'\n')
                       
    
f.close()

Finishing step: 0
Finishing step: 5000
Finishing step: 10000
Finishing step: 15000
Finishing step: 20000
Finishing step: 25000
Finishing step: 30000
Total Time 134.981365919
Pre loop Time 0.513035058975
Inner loop time 131.687339306
Outer loop time 2.50433468819
Reduced Dist 99.5615308285
out fn 5.64025974274
base_time 2.15268921852 2.55981230736
other time 21.0764698982 0.185108184814
Finishing step: 0
Finishing step: 5000
Finishing step: 10000
Finishing step: 15000
Finishing step: 20000
Finishing step: 25000
Finishing step: 30000
Total Time 77.9770519733
Pre loop Time 0.246728897095
Inner loop time 75.0600059032
Outer loop time 2.40628552437
Reduced Dist 11.7513456345
out fn 5.94490361214
base_time 4.68369054794 3.18126893044
other time 48.7140922546 0.301617383957
Finishing step: 0
Finishing step: 5000
Finishing step: 10000
Finishing step: 15000
Finishing step: 20000
Finishing step: 25000
Finishing step: 30000
Total Time 93.6265900135
Pre loop Time 0.000843048095703
Inner loop time 

In [3]:
g = open('workfileNR26full_uncorr.txt', 'w')
g.write('Northridge, %d stations \n' %(np.size(stationdata['pga'])))
s = str(variables['M'])
g.write(s+'\n')
s = str(variables['N'])
g.write(s+'\n')

start = time.time()
# Using the collected data, calculate correlation matrix
cor_model = BaseCorrelationModel
JB_cor_model = JB2009CorrelationModel(cor_model)

t2 = time.time() - start
comb_cor_model = JB_cor_model._get_correlation_matrix(variables['site_collection_both'], from_string(voi))
t3 = time.time() - start
# From Characterization of Uncertainty in Shakemaps, we have
# Sig11 = station_correlation_model
# Sig22 = grid_correlation_model
# Sig12 = Sig21^T

Sig11 = comb_cor_model[0:K, 0:K]
Sig12 = comb_cor_model[0:K, K:M*N+K]
Sig22 = comb_cor_model[K:M*N+K, K:M*N+K]

if np.size(Sig11) == 0:
    R = np.linalg.cholesky(Sig22)
    rand = np.reshape(rand, [M*N,1])
    X = R*rand
else:
    
    t4 = time.time()
    Sig11inv = linalg.pinv(Sig11)
    t5 = time.time() - t4
    # Xobs residuals
    x = np.zeros([K,1])

    # Calculate mean and covariance matrices specified in Jayaram and Baker 2009
    mean = np.array(Sig12.T * Sig11inv * x)
    cov = Sig22 - (Sig12.T*Sig11inv*Sig12)
    
    # Find distribution 
    t4 = time.time()
    R = np.linalg.cholesky(cov)
    t6 = time.time() - t4
    rand = np.reshape(rand, [M*N,1])
    X = mean + R*rand

COR = np.reshape(X, [M,N])

#Multiply by uncertainty
X = np.multiply(COR, variables['uncertaintydata'])

DATA_NEW = np.multiply(variables['data'],np.exp(X))

for i in range(0,M):
    for j in range(0,N):

        s = str(COR[i,j])
        g.write(s+' ')
g.close()

[[  1.00000000e+00   5.16471158e-01   2.66742504e-01 ...,   4.00714320e-21
    2.50101301e-21   1.55330215e-21]
 [  5.16471158e-01   1.00000000e+00   5.16471158e-01 ...,   6.38774193e-21
    4.00714320e-21   2.50101301e-21]
 [  2.66742504e-01   5.16471158e-01   1.00000000e+00 ...,   1.01294579e-20
    6.38774193e-21   4.00714320e-21]
 ..., 
 [  4.00714320e-21   6.38774193e-21   1.01294579e-20 ...,   1.00000000e+00
    4.99580710e-01   2.49580923e-01]
 [  2.50101301e-21   4.00714320e-21   6.38774193e-21 ...,   4.99580710e-01
    1.00000000e+00   4.99580710e-01]
 [  1.55330215e-21   2.50101301e-21   4.00714320e-21 ...,   2.49580923e-01
    4.99580710e-01   1.00000000e+00]]


In [6]:
!cp 'workfileNR26_uncorr.txt' '/Users/sverros/Documents/Testing/fileNR0_subset_correlated.txt'
#!cp 'workfileNR26full_uncorr.txt' '/Users/sverros/Documents/Testing/fileNR0full_correlated_c.txt'